In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
import re
import tensorflow as tf
import numpy as np
import string

from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pyplot as plt
import os

In [0]:
def plot_accuracy (history):
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.xlabel('epoch')
  plt.ylabel('accuracy')
  plt.legend(['test', 'train'], loc='upper left')
  plt.show()

def plot_loss (history):
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.legend(['test', 'train'], loc='upper left')
  plt.show()

In [0]:
!ls

In [0]:
train_tuple, test_tuple = [], []
leave = set(string.punctuation)

f_train = open("./../input/train.txt")
f_test = open("./../input/test.txt")

idx = 0
while (idx < 25000):
    s = f_train.readline()
    s = s.replace("<br />", ' ')
    s = (''.join(ch for ch in s if ch not in leave)).lower().split()
    if (len(s) == 0):
      print('shit')
    if (idx < 12500):
      train_tuple.append((s, 0))
    else:
      train_tuple.append((s, 1))
    idx += 1

idx = 0
while (idx < 25000):
    s = f_test.readline()
    s = s.replace("<br />", ' ')
    s = (''.join(ch for ch in s if ch not in leave)).lower().split()
    if (idx < 12500):
      test_tuple.append((s, 0))
    else:
      test_tuple.append((s, 1))
    idx += 1

f_train.close()
f_test.close()
print(train_tuple[0])

In [0]:
np.random.shuffle(train_tuple)
np.random.shuffle(test_tuple)

validation_tuple = train_tuple [ : 5000]
train_tuple = train_tuple [5000 : ]

print(np.shape(train_tuple))
print(np.shape(validation_tuple))
print(np.shape(test_tuple))

In [0]:
words = [train_tuple[i][0][j] for i in range(20000) for j in range(len(train_tuple[i][0]))]
vocab = sorted(list(set(words)))
vocab = set(vocab)

word_to_int = dict((word, idx) for idx, word in enumerate(vocab))
int_to_word = dict((idx, word) for idx, word in enumerate(vocab))

def encoding(ls):                             # encodes one review
  curr_review = ls
  curr_review_from_vocab = []
  # print("entry")
  for word in curr_review:
    if word in vocab:
      curr_review_from_vocab.append(word_to_int[word])
  # print("exit")
  ln = len(curr_review_from_vocab)
  for idx in range(200-ln):                   # padding with extra zeroes to for length 200
    curr_review_from_vocab.append(0.0)
  curr_review_from_vocab = curr_review_from_vocab[:200]
  review_np_array = np.array(curr_review_from_vocab, dtype=np.float32)
  return review_np_array


def get_encoding(list_of_reviews):
  seq, label = [], []
  for i in range(len(list_of_reviews)):
    review_np_array = encoding(list_of_reviews[i][0])
    seq.append(review_np_array)
    label.append(list_of_reviews[i][1])
  seq = np.asarray(seq)
  label = np.asarray(label)
  return seq, label


train_data, train_label = get_encoding(train_tuple)
validation_data, validation_label = get_encoding(validation_tuple)
test_data, test_label = get_encoding(test_tuple)


In [0]:

train_words = [train_tuple[i][0][j] for i in range(20000) for j in range(len(train_tuple[i][0]))]
test_words = [test_tuple[i][0][j] for i in range(25000) for j in range(len(test_tuple[i][0]))]
valid_words = [validation_tuple[i][0][j] for i in range(5000) for j in range(len(validation_tuple[i][0]))]

vocab_total = sorted(list(set(train_words + test_words + valid_words)))
vocab_total = set(vocab_total)

print(len(vocab_total))

In [0]:
import gensim
from gensim import models

from gensim.models import Word2Vec
from gensim import corpora

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer

english_stopwords = set(stopwords.words('english'))

def clean(data):
    train = []
    for v in data:                                       # one review (v)
        temp = []
        temp = [w for w in v[0] if not w in english_stopwords]
        # temp = np.asarray(temp)
        train.append(temp)                                 # train is a list of sentence (list of list of words)
    # train = np.asarray(train)
    return train


# print(len(train_data_1))
# print(train_data_1[0])

w2v_cbow = Word2Vec(size=128, window=5, min_count=5, alpha=0.01, min_alpha=0.001, workers=4)
w2v_sg = Word2Vec(size=128, window=5, min_count=5, alpha=0.01, min_alpha=0.001, workers=4, sg=1)

w2v_cbow_1 = Word2Vec(size=128, window=5, min_count=5, alpha=0.01, min_alpha=0.001, workers=4)



In [0]:
w2v_cbow.build_vocab(train_data_1 + test_data_1 + validation_data_1, progress_per=1000)
w2v_sg.build_vocab(train_data_1 + test_data_1 + validation_data_1, progress_per=1000)


In [0]:
w2v_cbow_1.build_vocab(train_data, progress_per=1000)


In [0]:
print (len(w2v_cbow.wv.vocab))
print (len(w2v_sg.wv.vocab))

NUM_WORDS = len(w2v_cbow.wv.vocab)
print (w2v_cbow)
print (w2v_sg)

In [0]:
# Train both models using the preprocessed training data

w2v_cbow.train(train_data_1 + test_data_1 + validation_data_1, total_examples=w2v_cbow.corpus_count, epochs=30, report_delay=1)
# w2v_sg.train(train_data_1 + test_data_1 + validation_data_1, total_examples=w2v_sg.corpus_count, epochs=30, report_delay=1)

In [0]:
# Train both models using the preprocessed training data

w2v_cbow_1.train(train_data_1, total_examples=w2v_cbow_1.corpus_count, epochs=30, report_delay=1)
# w2v_sg.train(train_data_1 + test_data_1 + validation_data_1, total_examples=w2v_sg.corpus_count, epochs=30, report_delay=1)

In [0]:
def review_embedding(model, review):
    vec = []
    for word in review:
        vec.append(model.wv[word])
    return vec

def embed_dataset(model, list_of_tuples):
    dataset_vec = []
    for review in list_of_tuples:
        dataset_vec.append(review_embedding(model, review))
    return dataset_vec

embedding_matrix = []
for word in w2v_cbow_1.wv.vocab:
    embedding_matrix.append(w2v_cbow_1.wv[word])

embedding_matrix = np.asarray(embedding_matrix)
print(np.shape(embedding_matrix))

def preprocess(list_tuple):
    dataset_vec = []
    for tup in list_tuple:
        one_review = []
        for word in tup[0]:
            if word in w2v_cbow_1.wv.vocab:
                one_review.append(w2v_cbow_1.wv[word])
        temp = []
        for i in range(200 - len(one_review)):
            temp.append([0 for _ in range(200)])
        dataset_vec.append(one_review + temp)
    return dataset_vec

train_data_1, test_data_1, validation_data_1 = [], [], []
train_data_1 = preprocess(train_tuple)
test_data_1 = preprocess(test_tuple)
validation_data_1 = preprocess(validation_tuple)



In [0]:
'''
embedding_layer = Embedding(NUM_WORDS, 128, 
                            weights=[embedding_matrix], 
                            input_length=200, 
                            trainable=False)
model_6.add(layers.Embedding(len(w2v_cbow_1.wv.vocab), 
                             128, input_length=200, 
                             weights=[embedding_matrix], 
                             trainable=False))

'''        
# Till now best model was Model 5

import tensorflow

model_6 = tensorflow.keras.models.Sequential()
model_6.add(layers.Dropout(0.25))
model_6.add(layers.GRU(200, activation='tanh', return_sequences=True))
model_6.add(layers.GRU(200, activation='tanh', return_sequences=True))
model_6.add(layers.GRU(200, activation='tanh', return_sequences=False))

model_6.add(layers.Dense(1))
model_6.add(layers.Activation('sigmoid'))
model_6.build((None, 200, 128))

model_6.summary()


In [0]:
model_6.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history_6 = model_6.fit(train_data, train_label, epochs=2, validation_data=(validation_data, validation_label))


In [0]:
score_6 = model_6.evaluate(test_data, test_label, verbose=0)
print(score_6)
score_6_ = model_6.evaluate(validation_data, validation_label, verbose=0)
print(score_6_)
score_6__ = model_6.evaluate(train_data, train_label, verbose=0)
print(score_6__)

In [1]:
import pandas as pd
import numpy as np

columns = []
columns.append('Training Accuracy (%)')
columns.append('Validation Accuracy (%)')
columns.append('Test Accuracy (%)')


df = pd.DataFrame({'Model': np.linspace(1, 9, 9)})
df = pd.concat([df, pd.DataFrame(np.random.randn(9, 5), columns=columns)],
               axis=1)

df.iloc[0, 1] = 
df.iloc[0, 2] = 
df.iloc[0, 3] = 

df.iloc[1, 1] = 63.46
df.iloc[1, 2] = 
df.iloc[1, 3] = 

df.iloc[2, 1] = 
df.iloc[2, 2] = 
df.iloc[2, 3] = 

df.iloc[3, 1] = 99.89
df.iloc[3, 2] = 87.64
df.iloc[3, 3] = 84.02

df.iloc[4, 1] = 99.71
df.iloc[4, 2] = 86.86
df.iloc[4, 3] = 83.58

df

SyntaxError: ignored